In [1]:
import copy
import os
import time
import random
import sys
import warnings

import numpy as np
import pandas as pd
from typing import Any


from hyperimpute.plugins.imputers import Imputers
from hyperimpute.utils.distributions import enable_reproducible_results
import hyperimpute.logger as log

from benchmark_imputation import evaluate_dataset_repeated_internal

from IPython.display import HTML, display
import tabulate

import json

warnings.filterwarnings('ignore')
enable_reproducible_results()

imputers = Imputers()
log.add(sink=sys.stderr, level="INFO")

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from pathlib import Path

def get_imputer():
    return imputers.get("hyperimpute", 
        optimizer = "simple"
    )

def save_results(fname, results):
    path = Path("tutorial_02_hyperimpute_example_results")
    path.mkdir(parents=True, exist_ok=True)

    out = path / fname
    
    with open(out, 'w') as outfile:
        json.dump(results, outfile)
        
def evaluate_dataset_repeated(
    name,
    X_raw,
    y,
    ref_methods=["mean", "sklearn_missforest", "sklearn_ice", 
                 "gain", "sinkhorn", "softimpute", "miracle", "miwae"],
    scenarios=["MNAR", "MCAR", "MAR"],
    miss_pct=[0.1, 0.3, 0.5, 0.7],
    n_iter=1,
    debug=False,
):
    results = evaluate_dataset_repeated_internal(
        name = name,
        evaluated_model = get_imputer(),
        X_raw = X_raw,
        y = y,
        ref_methods=ref_methods,
        scenarios=scenarios,
        miss_pct=miss_pct,
        n_iter=n_iter,
        debug=debug,
    )
    
    save_results(name, results)

## Sanity check in  debug mode

In [3]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat', header = None, sep ="\\t")

y = df[5]
X_raw = df.drop(columns = [5])

evaluate_dataset_repeated("airfoil_debug", X_raw, y,        
            scenarios =  ["MNAR"],      
            ref_methods=["mean", "miracle"],
            debug = True,
            n_iter = 1,
            miss_pct = [0.3]
)

> evaluation trial  0
  > eval  MNAR 0.3


[2022-01-13T12:34:26.576142+0200][353152][INFO]   > BO iter 0
[2022-01-13T12:34:26.969530+0200][353152][INFO]      >>> Column 0 <-- score -0.02207321259644897 <-- Model random_forest_regressor
[2022-01-13T12:34:27.392770+0200][353152][INFO]      >>> Column 1 <-- score -0.02305180517219732 <-- Model catboost_regressor
[2022-01-13T12:34:27.770646+0200][353152][INFO]      >>> Column 2 <-- score -0.03354103691649689 <-- Model catboost_regressor
[2022-01-13T12:34:31.111581+0200][353152][INFO]   > BO iter 1
[2022-01-13T12:34:31.537858+0200][353152][INFO]      >>> Column 3 <-- score -0.04553676254540827 <-- Model catboost_regressor
[2022-01-13T12:34:31.886386+0200][353152][INFO]      >>> Column 4 <-- score -0.0005044679073548755 <-- Model xgboost_regressor
[2022-01-13T12:34:32.296918+0200][353152][INFO]      >>> Column 2 <-- score -0.0019666368204545367 <-- Model catboost_regressor
[2022-01-13T12:34:35.606147+0200][353152][INFO]   > BO iter 2
[2022-01-13T12:34:36.060592+0200][353152][INFO]   

benchmark took  62.69257068634033
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,miracle
MNAR,0.3,0.212094,0.291261,0.265912




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,miracle
MNAR,0.3,0.0579122,0.368713,0.226537




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,miracle
MNAR,0.3,7.04437,5.78295,7.58695


# Datasets

In [3]:
import hyperimpute.logger as log

log.remove()

|           Dataset          |  Size |
|:--------------------------:|:-----:|
|     airfoil self noise     |  1503 |
|     blood transfusion      |  748  |
|  breast cancer diagnostic  |  569  |
|         california         | 20640 |
|   climate model crashes    |  540  |
|    concrete compression    |  1030 |
|       concrete slump       |  103  |
| connectionist bench sonar  |  208  |
|  connectionist bench vowel |  990  |
|            iris            |  150  |
|      wine quality red      |  1599 |
|     wine quality white     |  4899 |
|            yeast           |  1484 |

## Dataset: UCI Airfoil Self-Noise Data Set

https://archive.ics.uci.edu/ml/datasets/airfoil+self-noise


In [20]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat', header = None, sep ="\\t")

df

,0,1,2,3,4,5
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461
...,...,...,...,...,...,...
1498,2500,15.6,0.1016,39.6,0.052849,110.264
1499,3150,15.6,0.1016,39.6,0.052849,109.254
1500,4000,15.6,0.1016,39.6,0.052849,106.604
1501,5000,15.6,0.1016,39.6,0.052849,106.224


In [21]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("airfoil", X_raw, y)

benchmark took  1447.8613860607147
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.122367,0.2784,0.18893,0.232815,0.263831,0.24567,0.401612,0.231589,0.348342
MNAR,0.3,0.213909,0.291261,0.252147,0.273654,0.309422,0.273301,0.441961,0.264797,0.337668
MNAR,0.5,0.280072,0.293272,0.308962,0.308256,0.321773,0.291009,0.447677,0.439609,0.316822
MNAR,0.7,0.297985,0.29174,0.354823,0.372088,0.333904,0.30607,0.438434,0.401833,0.333571
MCAR,0.1,0.139127,0.296115,0.226713,0.248475,0.305383,0.279924,0.360982,0.253657,0.395555
MCAR,0.3,0.226347,0.285134,0.250022,0.258142,0.27669,0.276777,0.383641,0.25382,0.332991
MCAR,0.5,0.295629,0.289883,0.305056,0.28255,0.328869,0.294881,0.38761,0.279653,0.31488
MCAR,0.7,0.326228,0.285772,0.333089,0.30615,0.359656,0.301081,0.430316,0.304089,0.309513
MAR,0.1,0.145495,0.279909,0.249923,0.25837,0.355357,0.276793,0.318824,0.255003,0.322249
MAR,0.3,0.123435,0.286482,0.265178,0.267875,0.323366,0.261863,0.41271,0.262786,0.346196




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0116624,0.108778,0.0541458,0.0606058,0.068355,0.0432056,0.082809,0.0620731,0.0645997
MNAR,0.3,0.0651536,0.368713,0.182748,0.19861,0.24683,0.181335,0.478545,0.208629,0.214782
MNAR,0.5,0.143904,0.581678,0.292405,0.328493,0.405259,0.326532,0.774852,0.515539,0.38731
MNAR,0.7,0.463654,0.832606,0.403141,0.574857,0.570473,0.464435,1.05907,0.783409,0.462571
MCAR,0.1,0.0147015,0.118904,0.0585198,0.071384,0.0692387,0.0527207,0.0587238,0.0743689,0.0922373
MCAR,0.3,0.0702775,0.343884,0.148158,0.181616,0.214814,0.168007,0.171974,0.194062,0.193765
MCAR,0.5,0.166508,0.585558,0.322583,0.337868,0.337797,0.320533,0.327429,0.415923,0.378497
MCAR,0.7,0.461248,0.806786,0.54786,0.515501,0.632019,0.426611,1.03109,0.567926,0.497414
MAR,0.1,0.00853102,0.0653981,0.0437396,0.047544,0.0601801,0.0310152,0.0470675,0.0481498,0.0392381
MAR,0.3,0.0319989,0.205208,0.0994449,0.128565,0.15883,0.0882804,0.154491,0.127656,0.113765




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.67485,2.03634,1.4522,1.09325,0.981441,1.39504,2.01701,1.61832,3.02637
MNAR,0.3,5.49556,6.22452,4.74505,6.0459,5.34648,7.40481,7.45616,5.31637,8.38492
MNAR,0.5,12.2396,15.2353,13.1539,20.8675,19.8386,13.6531,14.3648,14.7497,16.0451
MNAR,0.7,26.7331,29.9819,26.3758,34.9711,14.3622,28.7188,29.9716,27.815,30.9317
MCAR,0.1,1.49543,1.18268,1.57599,1.50912,1.82114,1.01803,2.2119,1.29794,2.81256
MCAR,0.3,3.92663,6.27454,6.27252,4.79892,9.54228,8.20051,7.42995,6.19918,6.60059
MCAR,0.5,12.034,14.8783,15.4478,16.387,15.7688,17.0483,14.5492,10.2725,13.9875
MCAR,0.7,27.237,26.3317,21.1771,31.7299,28.1999,24.0723,30.7084,26.5489,18.8518
MAR,0.1,0.726232,1.97756,0.0576979,1.32021,2.74585,1.2087,1.9503,1.8398,2.92224
MAR,0.3,3.29572,6.59196,5.82984,6.01255,5.85653,6.62075,4.28058,3.17365,7.66179


## Dataset: UCI Blood Transfusion Service Center Data Set

https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data

In [22]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data')

df

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0
...,...,...,...,...,...
743,23,2,500,38,0
744,21,2,500,52,0
745,23,3,750,62,0
746,39,1,250,39,0


In [23]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("blood", X_raw, y)

benchmark took  1049.3795380592346
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.112128,0.200757,0.111717,0.165004,0.185018,0.147427,0.180977,0.151038,0.211856
MNAR,0.3,0.249333,0.17741,0.120483,0.153165,0.155142,0.170831,0.172436,0.157639,0.291419
MNAR,0.5,0.206649,0.172157,0.157956,0.151019,0.161026,0.190675,0.232461,0.159046,0.193591
MNAR,0.7,0.207289,0.175545,0.149068,0.160509,0.242823,0.194477,0.248746,0.18467,0.186372
MCAR,0.1,0.0963446,0.162605,0.0972446,0.106728,0.14927,0.140814,0.132243,0.10441,0.203049
MCAR,0.3,0.146395,0.166064,0.119022,0.120897,0.145657,0.162692,0.152319,0.120475,0.256612
MCAR,0.5,0.151289,0.153696,0.123045,0.130226,0.155809,0.169264,0.165895,0.129112,0.164969
MCAR,0.7,0.473636,0.162156,0.166583,0.173176,0.209281,0.189064,0.267025,0.158416,0.172689
MAR,0.1,0.156692,0.25398,0.149654,0.245392,0.245081,0.18703,0.231626,0.243431,0.287151
MAR,0.3,0.155334,0.244473,0.157458,0.197735,0.250739,0.215793,0.230094,0.18948,0.549227




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.00988405,0.0584245,0.0228112,0.0192036,0.0533327,0.0163965,0.0290568,0.0187562,0.0443485
MNAR,0.3,0.0751912,0.146506,0.0530473,0.0474073,0.0968617,0.0510214,0.0902851,0.0543525,0.156912
MNAR,0.5,0.160413,0.243065,0.130353,0.127302,0.158967,0.0855062,0.202779,0.163228,0.114796
MNAR,0.7,0.216235,0.329201,0.18145,0.200159,0.379377,0.148364,0.333456,0.329977,0.234247
MCAR,0.1,0.0112513,0.0480681,0.0144134,0.0171992,0.0425414,0.0146762,0.0175564,0.0188606,0.0252074
MCAR,0.3,0.0442366,0.136849,0.0532196,0.0568515,0.0925437,0.0461175,0.059231,0.0621411,0.0951262
MCAR,0.5,0.110104,0.221145,0.0867474,0.0907799,0.187631,0.0820492,0.145858,0.0998094,0.138283
MCAR,0.7,0.75114,0.32429,0.200548,0.209093,0.332126,0.128067,0.329762,0.30028,0.213359
MAR,0.1,0.0146136,0.0347266,0.0158501,0.0167809,0.0358256,0.0135448,0.0193984,0.0179427,0.0217428
MAR,0.3,0.043123,0.101418,0.0422782,0.0314505,0.103838,0.0461714,0.0742781,0.0387629,0.181711




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0,0,-0.00534759,0,0,-0.00534759,-0.00534759,0.00534759,0
MNAR,0.3,0,-0.00534759,0.00534759,0,0,0.00534759,0,0,0.00534759
MNAR,0.5,0,-0.00534759,0,0.0106952,-0.0160428,-0.00534759,0.00534759,0,0.00534759
MNAR,0.7,0.00534759,0,0.00534759,0.00534759,0.00534759,0.00534759,0,0,0.0106952
MCAR,0.1,0,-0.00534759,0,-0.00534759,0.00534759,0,0.0106952,0.00534759,0
MCAR,0.3,0.00534759,0,0,0.00534759,-0.00534759,0.00534759,0,0,0.0106952
MCAR,0.5,0,0,0,0.00534759,0,0.00534759,0,0,0
MCAR,0.7,0.00534759,0.00534759,0,-0.0106952,0,-0.00534759,0,0,0
MAR,0.1,0,0,0,0,0,0.00534759,-0.00534759,0.0106952,0
MAR,0.3,0,-0.00534759,-0.00534759,0,0.0106952,0.0106952,0,0,-0.0106952


## Dataset: Breast Cancer Wisconsin (Diagnostic)

https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)

In [24]:
from sklearn.datasets import load_breast_cancer

X_raw, y = load_breast_cancer(as_frame = True, return_X_y = True)

X_raw

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [25]:
evaluate_dataset_repeated("bc", X_raw, y)

benchmark took  2259.2533781528473
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0586489,0.164594,0.0831455,0.0559135,0.0755932,0.0846318,0.101242,0.0655974,0.206909
MNAR,0.3,0.0714401,0.169174,0.0833826,0.0599373,0.0869396,0.0880044,0.109784,0.0754324,0.239128
MNAR,0.5,0.101167,0.16228,0.0754941,0.0545301,0.0865836,0.085696,0.101965,0.0766679,0.283477
MNAR,0.7,0.088392,0.158235,0.0662062,0.0448672,0.0837899,0.0792931,0.0999905,0.114977,0.230473
MCAR,0.1,0.0405549,0.156038,0.0629747,0.0412613,0.0737464,0.06685,0.101851,0.0476767,0.19067
MCAR,0.3,0.0367153,0.165539,0.0566628,0.0302051,0.0591266,0.0589224,0.0948663,0.0466679,0.237247
MCAR,0.5,0.070222,0.133916,0.0642796,0.041619,0.0629071,0.0681313,0.086996,0.0653489,0.153386
MCAR,0.7,0.131838,0.166575,0.0837292,0.06518,0.099459,0.0908587,0.112814,0.127247,0.265983
MAR,0.1,0.0400013,0.139049,0.0515367,0.0405673,0.060213,0.0671455,0.088019,0.043552,0.192341
MAR,0.3,0.0491763,0.140034,0.0633482,0.0424755,0.109448,0.0817756,0.110255,0.0608247,0.23271




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0133869,0.0961422,0.0289536,0.0132709,0.0272178,0.0240813,0.0354247,0.0218586,0.0494895
MNAR,0.3,0.0304968,0.289567,0.0728061,0.0311423,0.0783129,0.0668419,0.092804,0.0751347,0.163993
MNAR,0.5,0.0705295,0.443799,0.0933565,0.0422847,0.128518,0.102841,0.119289,0.120293,0.453561
MNAR,0.7,0.104675,0.6301,0.125888,0.0508342,0.219787,0.158523,0.238926,0.395756,0.393981
MCAR,0.1,0.00872372,0.0920457,0.0197359,0.00914363,0.0284735,0.0185343,0.0258735,0.0156699,0.0379077
MCAR,0.3,0.0166984,0.309434,0.0470086,0.0147358,0.0546506,0.0431547,0.0848641,0.0610029,0.139962
MCAR,0.5,0.0695435,0.375954,0.0826453,0.0390062,0.0836492,0.0759658,0.151245,0.113934,0.0965058
MCAR,0.7,0.146933,0.723716,0.126299,0.0965751,0.225065,0.181785,0.184289,0.476772,0.375973
MAR,0.1,0.00436811,0.0403042,0.0102702,0.00446001,0.00919205,0.00937689,0.0158056,0.00817791,0.0225782
MAR,0.3,0.0121242,0.111394,0.0204217,0.0106303,0.0421115,0.0312894,0.0448053,0.0240554,0.0911983




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.00699301,-0.013986,0.00699301,0,0.013986,0,-0.00699301,0.013986,0
MNAR,0.3,0.00699301,-0.00699301,0.00699301,0,-0.00699301,-0.00699301,0,-0.013986,0
MNAR,0.5,0.00699301,-0.00699301,-0.00699301,0.00699301,-0.00699301,0,0,0.00699301,-0.00699301
MNAR,0.7,0,0.00699301,-0.013986,0.013986,-0.00699301,0,0,0,0.00699301
MCAR,0.1,0,0,0,-0.00699301,-0.00699301,0.00699301,-0.013986,0,0
MCAR,0.3,-0.00699301,-0.013986,0,-0.00699301,-0.00699301,-0.013986,0,0.00699301,0.00699301
MCAR,0.5,-0.00699301,0.013986,0.00699301,0,0,-0.00699301,-0.00699301,-0.013986,0
MCAR,0.7,0,-0.020979,0.00699301,-0.00699301,-0.00699301,-0.00699301,0.013986,0,-0.00699301
MAR,0.1,0,0,-0.00699301,0,-0.020979,0.00699301,0.00699301,0,-0.027972
MAR,0.3,-0.00699301,0.013986,0.00699301,0.00699301,0,0,-0.00699301,0.013986,0.034965


## Dataset: California Housing



In [26]:
from sklearn.datasets import fetch_california_housing

X_raw, y = fetch_california_housing(as_frame = True, return_X_y = True)

X_raw

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [27]:
evaluate_dataset_repeated("california", X_raw, y)

benchmark took  13518.93718123436
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0813701,0.144448,0.0977407,0.0967474,0.134907,0.154023,0.178484,0.103947,0.150992
MNAR,0.3,0.129961,0.145019,0.117986,0.213821,0.192512,0.172506,0.215511,0.129763,0.152777
MNAR,0.5,0.157329,0.1461,0.145623,0.160333,0.173133,0.181157,0.267132,0.136083,0.144726
MNAR,0.7,0.203955,0.152515,0.16115,0.597297,0.266495,0.189896,0.256033,0.182146,0.164386
MCAR,0.1,0.0835303,0.145577,0.0980449,0.0980449,0.133301,0.151771,0.183151,0.103635,0.152434
MCAR,0.3,0.118646,0.146846,0.119007,0.126923,0.144227,0.17457,0.204598,0.122431,0.141592
MCAR,0.5,0.167946,0.146218,0.133254,0.14257,0.181285,0.181198,0.225889,0.131233,0.153878
MCAR,0.7,0.211941,0.146396,0.167841,0.145388,0.285662,0.184264,0.281147,0.153733,0.154367
MAR,0.1,0.0875351,0.180765,0.112479,0.170742,0.151201,0.194626,0.20744,0.13243,0.190284
MAR,0.3,0.104999,0.175527,0.121823,0.180601,0.152302,0.18557,0.225516,0.13654,0.143647




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.00788032,0.0721136,0.0242266,0.0222481,0.0479157,0.0262874,0.0472424,0.0317527,0.0528314
MNAR,0.3,0.029427,0.217728,0.0954968,0.0934778,0.236466,0.0737236,0.107162,0.144633,0.151265
MNAR,0.5,0.0899668,0.361934,0.185979,0.166296,0.336703,0.140393,0.363048,0.286745,0.168593
MNAR,0.7,0.318419,0.517079,0.302361,0.37286,0.909095,0.301612,0.612421,0.565921,0.383214
MCAR,0.1,0.00805114,0.0720225,0.0238616,0.0216447,0.0525656,0.0207456,0.0420291,0.0300229,0.0507232
MCAR,0.3,0.0238189,0.218013,0.082586,0.0815205,0.150132,0.0678078,0.12303,0.147213,0.0784554
MCAR,0.5,0.0930918,0.362035,0.216716,0.184568,0.325009,0.134584,0.28164,0.271377,0.253855
MCAR,0.7,0.358555,0.508492,0.312688,0.427995,0.965866,0.237051,0.659237,0.526027,0.355182
MAR,0.1,0.00579414,0.0531253,0.0163751,0.0160842,0.035055,0.0185143,0.0346201,0.025352,0.0410007
MAR,0.3,0.0177361,0.131059,0.0573615,0.0454579,0.0922501,0.0687182,0.0768065,0.0836125,0.0523573




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0145311,0.016885,0.0178677,0.0259019,0.029686,0.0279931,0.033003,0.0226845,0.0236202
MNAR,0.3,0.0871524,0.0636721,0.0730051,0.071627,0.0673217,0.113922,0.0638952,0.0605952,0.0855458
MNAR,0.5,0.190145,0.14004,0.104414,0.151435,0.171776,0.180648,0.159501,0.144387,0.159584
MNAR,0.7,0.409325,0.358834,0.251599,0.254931,0.195184,0.383908,0.236014,0.226189,0.275825
MCAR,0.1,0.0225601,0.0107601,0.00986001,0.0193389,0.0268876,0.0242221,0.0287112,0.0195073,0.0360723
MCAR,0.3,0.0722354,0.0670141,0.0516594,0.0656888,0.0681838,0.0862519,0.0732818,0.0518113,0.0777696
MCAR,0.5,0.16828,0.132259,0.128507,0.152595,0.119622,0.187456,0.122967,0.10207,0.157912
MCAR,0.7,0.292266,0.318494,0.247701,0.253885,0.219468,0.339667,0.24291,0.249127,0.285877
MAR,0.1,0.00724193,0.0068212,-0.000191954,0.00746806,0.0134129,0.0159503,0.0157306,0.0014695,0.0118893
MAR,0.3,0.0337848,0.0278388,0.0360647,0.0467918,0.044094,0.0452227,0.0510075,0.0456122,0.0393889


## Dataset: Climate Model Simulation Crashes
https://archive.ics.uci.edu/ml/datasets/climate+model+simulation+crashes

In [28]:
samples = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/00252/pop_failures.dat", skiprows=1)
df = pd.DataFrame(samples)

df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1.0,1.0,0.859036,0.927825,0.252866,0.298838,0.170521,0.735936,0.428325,0.567947,...,0.245675,0.104226,0.869091,0.997518,0.448620,0.307522,0.858310,0.796997,0.869893,0.0
1,1.0,2.0,0.606041,0.457728,0.359448,0.306957,0.843331,0.934851,0.444572,0.828015,...,0.616870,0.975786,0.914344,0.845247,0.864152,0.346713,0.356573,0.438447,0.512256,1.0
2,1.0,3.0,0.997600,0.373238,0.517399,0.504993,0.618903,0.605571,0.746225,0.195928,...,0.679355,0.803413,0.643995,0.718441,0.924775,0.315371,0.250642,0.285636,0.365858,1.0
3,1.0,4.0,0.783408,0.104055,0.197533,0.421837,0.742056,0.490828,0.005525,0.392123,...,0.471463,0.597879,0.761659,0.362751,0.912819,0.977971,0.845921,0.699431,0.475987,1.0
4,1.0,5.0,0.406250,0.513199,0.061812,0.635837,0.844798,0.441502,0.191926,0.487546,...,0.551543,0.743877,0.312349,0.650223,0.522261,0.043545,0.376660,0.280098,0.132283,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,3.0,176.0,0.657136,0.489375,0.133713,0.411950,0.087780,0.356289,0.480204,0.029678,...,0.280546,0.384117,0.885948,0.768482,0.459479,0.334482,0.573002,0.610183,0.737706,1.0
536,3.0,177.0,0.915894,0.842720,0.518947,0.090622,0.336981,0.893576,0.978703,0.674868,...,0.798108,0.353546,0.044796,0.990900,0.347027,0.512499,0.810549,0.593332,0.142565,0.0
537,3.0,178.0,0.478600,0.941185,0.769245,0.950776,0.189406,0.112743,0.745645,0.527096,...,0.193103,0.829563,0.101506,0.548878,0.381966,0.198811,0.867108,0.461632,0.652817,1.0
538,3.0,179.0,0.007793,0.779287,0.867468,0.704820,0.983282,0.420303,0.710612,0.174746,...,0.761134,0.436714,0.690132,0.825133,0.981656,0.113193,0.364799,0.201469,0.536535,1.0


In [29]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("climate", X_raw, y)

benchmark took  1650.71315908432
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.357814,0.301557,0.310289,0.301615,0.380576,0.357627,0.33866,0.302443,0.366621
MNAR,0.3,0.303211,0.289646,0.30374,0.290702,0.351635,0.328789,0.30015,0.289584,0.324945
MNAR,0.5,0.33031,0.290688,0.330421,0.290858,0.360905,0.325853,0.300454,0.298907,0.343998
MNAR,0.7,0.395434,0.306848,0.381229,0.307258,0.39775,0.328151,0.336641,0.477866,0.373484
MCAR,0.1,0.301111,0.294602,0.298597,0.294735,0.338286,0.337221,0.302973,0.294505,0.375774
MCAR,0.3,0.305929,0.290752,0.305804,0.290837,0.346888,0.336041,0.30226,0.291226,0.361886
MCAR,0.5,0.396735,0.305302,0.3495,0.305684,0.370338,0.340914,0.349568,0.307746,0.419816
MCAR,0.7,0.405833,0.308038,0.395856,0.308188,0.369298,0.339039,0.337134,0.326801,0.42654
MAR,0.1,0.310483,0.307304,0.31215,0.307223,0.377268,0.356363,0.315214,0.311819,0.393695
MAR,0.3,0.304365,0.292668,0.309041,0.292735,0.364275,0.326957,0.300468,0.294392,0.361277




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.197859,0.225693,0.198659,0.225541,0.163898,0.1247,0.184026,0.223122,0.111367
MNAR,0.3,0.439,0.612733,0.437711,0.598671,0.299592,0.337337,0.481169,0.612861,0.361954
MNAR,0.5,0.523786,1.01299,0.511981,1.00464,0.584249,0.58465,0.777063,0.92459,0.498518
MNAR,0.7,0.677721,1.44966,0.570311,1.43457,0.994103,0.983584,1.03822,1.72283,0.625865
MCAR,0.1,0.174543,0.201929,0.17701,0.20053,0.120809,0.102901,0.162459,0.201218,0.0871092
MCAR,0.3,0.472118,0.606451,0.468402,0.605641,0.295488,0.326837,0.47346,0.607049,0.231136
MCAR,0.5,0.449752,1.05096,0.58585,1.03638,0.519427,0.59572,0.706339,1.05892,0.877062
MCAR,0.7,0.503959,1.47369,0.541296,1.46526,0.79845,1.00473,1.08131,1.55158,1.20208
MAR,0.1,0.0938733,0.106357,0.0931962,0.104872,0.0747181,0.0556981,0.0865312,0.100143,0.0507523
MAR,0.3,0.24154,0.310674,0.235981,0.310313,0.216318,0.189714,0.247305,0.31116,0.134135




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0,0,0,0,0,0,0,0,0
MNAR,0.3,0,0,0,0,0,0,0,0,0
MNAR,0.5,0,0,0,0,0,0,0,0,0
MNAR,0.7,0,0,0,0,0,0,0,0,0
MCAR,0.1,0,0,0,0,0,0,0,0,0
MCAR,0.3,0,0,0,0,0,0,0,0,0
MCAR,0.5,0,0,0,0,0,0,0,0,0
MCAR,0.7,0,0,0,0,0,0,0,0,0
MAR,0.1,0,0,0,0,0,0,0,0,0
MAR,0.3,0,0,0,0,0,0,0,0,0


## Concrete Compressive Strength Data Set
https://archive.ics.uci.edu/ml/datasets/concrete+compressive+strength

In [12]:
df = pd.read_excel("https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls")

df

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.284354
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.178794
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.696601
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.768036


In [13]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("compression", X_raw, y)

benchmark took  1524.1037979125977
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0993771,0.227739,0.153854,0.140717,0.219689,0.150478,0.256924,0.144949,0.245225
MNAR,0.3,0.155639,0.224163,0.195844,0.179739,0.23177,0.180147,0.274853,0.198206,0.241737
MNAR,0.5,0.226426,0.228344,0.223043,0.21111,0.233067,0.211636,0.324197,0.263522,0.267456
MNAR,0.7,0.255833,0.227936,0.252596,0.21972,0.284702,0.247878,0.357194,0.320076,0.32571
MCAR,0.1,0.0898357,0.21943,0.149057,0.130554,0.203618,0.141538,0.250211,0.131507,0.242495
MCAR,0.3,0.139154,0.222247,0.174821,0.171198,0.218309,0.181873,0.270317,0.190611,0.258876
MCAR,0.5,0.214683,0.222449,0.224208,0.204604,0.247798,0.210051,0.29962,0.246214,0.237334
MCAR,0.7,0.342977,0.225954,0.24879,0.221403,0.352594,0.244664,0.338825,0.338087,0.240998
MAR,0.1,0.049912,0.208818,0.137668,0.106967,0.196171,0.0912681,0.23152,0.111372,0.22243
MAR,0.3,0.112762,0.244211,0.17834,0.139626,0.234685,0.134681,0.312483,0.147181,0.262899




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0215753,0.137191,0.0552282,0.0390588,0.0861911,0.0211041,0.0779951,0.0444887,0.094422
MNAR,0.3,0.0871342,0.437166,0.191749,0.169741,0.305622,0.100507,0.208764,0.220971,0.286532
MNAR,0.5,0.227246,0.742986,0.349016,0.426426,0.494087,0.265496,0.338689,0.539207,0.359631
MNAR,0.7,0.406594,1.02787,0.473797,0.737472,0.926254,0.438443,0.677489,1.1514,0.67764
MCAR,0.1,0.0180551,0.139828,0.0503694,0.0389706,0.0991717,0.0233192,0.0824483,0.0430421,0.083898
MCAR,0.3,0.0604206,0.437029,0.158206,0.163264,0.245046,0.0897424,0.197592,0.216001,0.222523
MCAR,0.5,0.23205,0.710977,0.299002,0.415571,0.519397,0.199188,0.306998,0.475836,0.464174
MCAR,0.7,0.637877,1.01018,0.451562,0.734879,1.3451,0.383979,0.599061,1.20891,0.655513
MAR,0.1,0.00512423,0.0671591,0.0279967,0.0151808,0.0499697,0.0104344,0.0320943,0.0188305,0.0465432
MAR,0.3,0.0429734,0.260642,0.10483,0.088293,0.170882,0.0367008,0.161255,0.105369,0.168129




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,9.87367,16.1105,8.80637,10.0738,7.42299,9.95732,11.4532,8.54534,12.6429
MNAR,0.3,15.021,59.4063,37.5257,27.846,25.9482,31.4976,27.4738,25.4375,38.5767
MNAR,0.5,54.0868,99.9873,79.5432,41.9518,49.291,59.4671,72.0233,54.4669,57.0673
MNAR,0.7,110.36,191.686,129.022,112.704,94.1723,121.919,100.301,94.3956,129.382
MCAR,0.1,3.30562,13.5928,6.22976,0.501382,6.30072,6.01313,9.57304,1.64734,7.70075
MCAR,0.3,11.2614,26.7743,17.8826,18.857,34.9135,15.4934,30.1869,26.3358,28.9649
MCAR,0.5,64.7886,77.603,64.5132,55.4487,53.5576,46.4627,62.3332,63.6179,84.4107
MCAR,0.7,182.139,157.73,125.312,114.205,84.7526,102.076,138.695,99.959,119.778
MAR,0.1,3.1689,2.08376,3.25633,-1.13268,1.44604,1.13328,4.10265,2.19609,6.09528
MAR,0.3,-1.41529,8.51288,5.00876,4.81919,5.85844,2.56558,5.11963,1.77082,6.5161


## Concrete Slump Test Data Set

https://archive.ics.uci.edu/ml/datasets/concrete+slump+test

In [14]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/slump/slump_test.data")

df

,No,Cement,Slag,Fly ash,Water,SP,Coarse Aggr.,Fine Aggr.,SLUMP(cm),FLOW(cm),Compressive Strength (28-day)(Mpa)
0,1,273.0,82.0,105.0,210.0,9.0,904.0,680.0,23.0,62.0,34.99
1,2,163.0,149.0,191.0,180.0,12.0,843.0,746.0,0.0,20.0,41.14
2,3,162.0,148.0,191.0,179.0,16.0,840.0,743.0,1.0,20.0,41.81
3,4,162.0,148.0,190.0,179.0,19.0,838.0,741.0,3.0,21.5,42.08
4,5,154.0,112.0,144.0,220.0,10.0,923.0,658.0,20.0,64.0,26.82
...,...,...,...,...,...,...,...,...,...,...,...
98,99,248.3,101.0,239.1,168.9,7.7,954.2,640.6,0.0,20.0,49.97
99,100,248.0,101.0,239.9,169.1,7.7,949.9,644.1,2.0,20.0,50.23
100,101,258.8,88.0,239.6,175.3,7.6,938.9,646.0,0.0,20.0,50.50
101,102,297.1,40.9,239.9,194.0,7.5,908.9,651.8,27.5,67.0,49.17


In [15]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = ["No", last_col])

evaluate_dataset_repeated("slump", X_raw, y)

benchmark took  849.2699918746948
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.152858,0.316456,0.251852,0.166722,0.289322,0.266929,0.343033,0.813947,0.484408
MNAR,0.3,0.254397,0.28564,0.242575,0.223173,0.263613,0.241835,0.330368,0.473239,0.385229
MNAR,0.5,0.290588,0.298055,0.297495,0.268063,0.337183,0.280485,0.334239,0.342464,0.393772
MNAR,0.7,0.326701,0.288191,0.3325,0.269075,0.432123,0.274987,0.3895,0.573077,0.298603
MCAR,0.1,0.213636,0.28187,0.295867,0.202357,0.265199,0.259185,0.309622,0.205702,0.293
MCAR,0.3,0.244976,0.275632,0.20489,0.173422,0.231509,0.204901,0.283465,0.683016,0.340056
MCAR,0.5,0.325919,0.2865,0.287828,0.239279,0.336654,0.257268,0.360535,0.303764,0.367063
MCAR,0.7,0.360734,0.283459,0.340704,0.29178,0.482482,0.286128,0.356965,0.643406,0.34366
MAR,0.1,0.152921,0.305788,0.221577,0.1052,0.259883,0.199362,0.30991,0.116395,0.356583
MAR,0.3,0.200751,0.28411,0.196996,0.16007,0.232742,0.220042,0.309364,0.236802,0.327307




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0510251,0.198258,0.110091,0.0620045,0.121409,0.110041,0.124105,0.205523,0.209778
MNAR,0.3,0.167398,0.582991,0.273564,0.261029,0.380545,0.272617,0.350805,0.367682,0.416231
MNAR,0.5,0.487283,1.0295,0.39691,0.544727,0.807762,0.55971,0.467434,0.706204,0.593379
MNAR,0.7,0.692256,1.33863,0.614475,0.821977,1.45277,0.737192,0.563667,1.8632,0.873611
MCAR,0.1,0.0581012,0.162582,0.0823211,0.0610685,0.0964412,0.0911298,0.0835794,0.0621813,0.115405
MCAR,0.3,0.20175,0.557734,0.241639,0.192494,0.282803,0.233553,0.292563,0.428682,0.290028
MCAR,0.5,0.409045,0.952328,0.438434,0.496744,0.80339,0.484277,0.488084,0.55911,0.438966
MCAR,0.7,0.567005,1.36893,0.603566,0.821113,2.17106,0.785132,0.688557,2.10381,0.807778
MAR,0.1,0.0255281,0.104451,0.0493059,0.0230566,0.0502185,0.0371543,0.0658591,0.0284114,0.0653559
MAR,0.3,0.0995965,0.283186,0.116926,0.096544,0.160832,0.124334,0.170414,0.123572,0.137518




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,-0.871941,4.08369,2.898,0.687763,-8.45577,0.714016,5.8522,-3.47784,3.78639
MNAR,0.3,6.4367,3.57767,9.09925,12.897,7.17501,1.29118,8.41047,4.1116,14.8018
MNAR,0.5,17.7465,17.8963,23.0909,29.8143,17.5884,29.3754,49.5639,25.3564,22.5866
MNAR,0.7,44.1496,60.5035,77.9285,34.2466,86.7366,53.9895,61.9406,43.0386,63.6507
MCAR,0.1,-0.104416,6.32341,1.70438,-0.213802,-0.440855,-1.2919,0.00660181,-9.11337,5.36485
MCAR,0.3,3.76058,11.7757,7.58723,17.343,8.0333,5.90236,5.45278,19.7325,4.03078
MCAR,0.5,-0.0413202,27.0336,-5.61866,9.80803,8.5572,5.86743,26.8403,19.5046,24.0133
MCAR,0.7,44.9443,44.8283,12.6023,53.9296,53.9298,29.9261,94.2782,20.8655,17.5089
MAR,0.1,1.48448,0.304534,0.108207,0.742513,0.991525,-1.9161,-0.486837,1.02523,-1.72089
MAR,0.3,3.76666,4.06447,-3.32538,3.5102,1.73128,-0.711867,8.92184,5.24707,20.3682


## Connectionist Bench (Sonar, Mines vs. Rocks) Data Set

https://archive.ics.uci.edu/ml/datasets/connectionist+bench+(sonar,+mines+vs.+rocks)

In [16]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data", header = None)

df

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


In [17]:
last_col = df.columns[-1]
y = (df[last_col] == 'M').astype(int)
X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("sonar", X_raw, y)

benchmark took  2382.5324289798737
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.126733,0.217123,0.127985,0.120214,0.145891,0.149458,0.21115,0.121212,0.421007
MNAR,0.3,0.104996,0.212844,0.101372,0.0881613,0.118646,0.126129,0.176182,0.200783,0.285902
MNAR,0.5,0.113933,0.211549,0.113185,0.113917,0.136968,0.155242,0.183889,0.196464,0.296954
MNAR,0.7,0.126613,0.194971,0.127946,0.120791,0.184856,0.15269,0.186243,0.201714,0.212983
MCAR,0.1,0.0975227,0.218385,0.102767,0.0896417,0.12637,0.147471,0.200351,0.0837329,0.309092
MCAR,0.3,0.11301,0.213458,0.111419,0.0994409,0.125679,0.132943,0.171648,0.0979958,0.238783
MCAR,0.5,0.117807,0.187969,0.116052,0.114442,0.123943,0.130193,0.165917,0.368918,0.272885
MCAR,0.7,0.131561,0.202424,0.128999,0.122166,0.15283,0.142867,0.178055,0.559859,0.220589
MAR,0.1,0.0983848,0.218539,0.092851,0.0846221,0.109801,0.126702,0.208966,0.0786057,0.256535
MAR,0.3,0.123499,0.244537,0.126884,0.102185,0.161487,0.162802,0.214687,0.268926,0.326733




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0435392,0.144021,0.042777,0.0386515,0.0534366,0.0582535,0.0980384,0.0331729,0.158144
MNAR,0.3,0.109287,0.39916,0.09684,0.0632783,0.113374,0.14938,0.177124,0.248529,0.200066
MNAR,0.5,0.163954,0.664946,0.174818,0.118348,0.222028,0.310691,0.332903,0.573079,0.28762
MNAR,0.7,0.281925,0.822441,0.243142,0.227766,0.424212,0.458428,0.519624,0.805795,0.446712
MCAR,0.1,0.0274057,0.12421,0.0381428,0.0261935,0.0417951,0.0533017,0.0739373,0.0267289,0.0584694
MCAR,0.3,0.126023,0.42475,0.123262,0.090623,0.104755,0.15163,0.191821,0.0874054,0.257594
MCAR,0.5,0.130822,0.568488,0.126788,0.107345,0.150048,0.257258,0.346966,0.814971,0.226095
MCAR,0.7,0.239154,0.832808,0.239077,0.15147,0.245306,0.391839,0.44862,1.84894,0.505538
MAR,0.1,0.0177109,0.0677466,0.0188133,0.0150733,0.0254297,0.0279446,0.0406199,0.0144185,0.0433352
MAR,0.3,0.0539791,0.227154,0.0597792,0.0424947,0.0679442,0.101865,0.136031,0.234017,0.172583




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,-0.0192308,0.0384615,0.0384615,-0.0384615,-0.0576923,0.0192308,0.0384615,0,0
MNAR,0.3,0.0192308,-0.0192308,0,0.0192308,0.0192308,-0.0576923,-0.0384615,0.0384615,0.0384615
MNAR,0.5,-0.0192308,0,0.0192308,-0.0192308,0,0.0192308,-0.0192308,0.0576923,0.0192308
MNAR,0.7,0,-0.0192308,0,0,-0.0576923,0,0.0384615,0,0.0192308
MCAR,0.1,0.0576923,0.0576923,0,0.0384615,0.0192308,0.0384615,0,0.0192308,0.0384615
MCAR,0.3,0.0192308,-0.0192308,0.0384615,-0.0576923,-0.0192308,-0.0192308,0.0192308,0.0192308,-0.0192308
MCAR,0.5,-0.0384615,-0.0576923,0,0.0192308,0.0769231,-0.0384615,0.0769231,-0.0384615,-0.0192308
MCAR,0.7,0.0192308,0.0384615,-0.0192308,0,0.0384615,-0.0576923,0.0384615,0.0769231,0.0576923
MAR,0.1,-0.0192308,0.0192308,0,0.0192308,0.0192308,0.0192308,0.134615,0.0192308,0.0192308
MAR,0.3,0.0192308,-0.0192308,-0.0192308,-0.0961538,-0.0384615,0.0384615,0.0192308,0.0576923,0.0192308


## Wine-Red dataset

In [18]:
# Wine Quality Data Set

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep = ';')

df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [19]:
last_col = df.columns[-1]

y = df[last_col]
mapped_labels = sorted(y.unique())
mapping = {}
for idx, label in enumerate(mapped_labels):
    mapping[label] = idx
y = y.map(mapping)       

X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("wine_red", X_raw, y)

benchmark took  2187.453905105591
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0834124,0.144911,0.10811,0.101544,0.124248,0.1007,0.135421,0.104636,0.223669
MNAR,0.3,0.107968,0.154177,0.117731,0.115615,0.14111,0.116165,0.150252,0.11568,0.189483
MNAR,0.5,0.118877,0.120408,0.103278,0.112065,0.139757,0.0996507,0.130682,0.105489,0.128703
MNAR,0.7,0.15044,0.135106,0.133479,0.114258,0.173873,0.118223,0.137992,0.149128,0.143171
MCAR,0.1,0.0710448,0.12884,0.0953694,0.0851187,0.106838,0.0897577,0.12347,0.0911515,0.177648
MCAR,0.3,0.0829912,0.124938,0.0934157,0.0970738,0.108909,0.0912113,0.121894,0.0931941,0.132937
MCAR,0.5,0.112521,0.141398,0.117747,0.119833,0.147404,0.115916,0.140209,0.115869,0.1938
MCAR,0.7,0.192802,0.12956,0.135057,0.106889,0.138636,0.114044,0.13135,0.136288,0.175612
MAR,0.1,0.0658882,0.130743,0.0918323,0.0877899,0.110362,0.0845681,0.137696,0.0914326,0.14236
MAR,0.3,0.0717497,0.136602,0.0909916,0.0887511,0.121613,0.0856411,0.128739,0.093302,0.144192




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.014583,0.0853732,0.0382662,0.0252974,0.0456583,0.0198328,0.0461525,0.034025,0.0650381
MNAR,0.3,0.0505138,0.282515,0.104943,0.0804998,0.166767,0.0733401,0.153653,0.131172,0.113676
MNAR,0.5,0.10239,0.331229,0.137117,0.0993881,0.324193,0.0961735,0.156512,0.202993,0.202941
MNAR,0.7,0.192201,0.539057,0.269997,0.256921,0.487873,0.170382,0.3568,0.535333,0.344752
MCAR,0.1,0.0131145,0.0718677,0.029997,0.0203472,0.036519,0.0158258,0.0372564,0.0328847,0.038185
MCAR,0.3,0.0359817,0.213124,0.0862797,0.0534361,0.101881,0.047094,0.119976,0.0922541,0.13009
MCAR,0.5,0.101649,0.41842,0.149517,0.134139,0.330776,0.100775,0.210666,0.20874,0.186693
MCAR,0.7,0.32189,0.516801,0.19344,0.235482,0.422145,0.170065,0.339096,0.463962,0.252874
MAR,0.1,0.00657446,0.0369488,0.0135128,0.00866353,0.020487,0.00940207,0.0169503,0.0119804,0.0202059
MAR,0.3,0.0212664,0.117755,0.041007,0.0286931,0.0643231,0.0282162,0.0596649,0.0478669,0.073005




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,-0.0275,-0.01,0.0575,0.0125,0.0275,-0.04,-0.0425,-0.02,0.04
MNAR,0.3,-0.0075,0.005,-0.0125,-0.0075,0.0225,-0.0025,0.06,-0.005,0.0625
MNAR,0.5,0,-0.0025,0.015,-0.02,0.07,0.035,0.0425,0.005,0.0075
MNAR,0.7,0.085,0.1525,-0.005,0.105,0.07,-0.0375,0.02,0.02,0.0425
MCAR,0.1,0.01,-0.065,-0.05,0.0075,-0.04,0.045,0.02,0.015,0.0625
MCAR,0.3,0.01,-0.035,-0.0075,0.015,0.065,-0.0025,0.01,0.0475,0.035
MCAR,0.5,0.0175,0,0.035,-0.025,0.0375,0.0025,0.07,0.035,-0.03
MCAR,0.7,-0.015,0.005,0.085,-0.03,-0.0075,0.02,0.0075,-0.0075,0.105
MAR,0.1,0.0225,0.015,-0.0125,0.0375,0.0225,-0.0175,-0.0125,0.015,-0.0375
MAR,0.3,0.005,0.045,0.025,-0.0275,0.0275,0.02,0.0575,0.015,0.015


## Wine-White dataset

In [8]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep = ';')

df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [9]:
last_col = df.columns[-1]

y = df[last_col]
mapped_labels = sorted(y.unique())
mapping = {}
for idx, label in enumerate(mapped_labels):
    mapping[label] = idx
y = y.map(mapping)  

X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("wine_white", X_raw, y)

benchmark took  4502.226128816605
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0773425,0.0957049,0.0839042,0.0770585,0.0919062,0.0971846,0.100697,0.0847709,0.110677
MNAR,0.3,0.0777224,0.108291,0.0852112,0.0802788,0.10766,0.103153,0.119734,0.0836326,0.113043
MNAR,0.5,0.09554,0.106531,0.0952918,0.093921,0.166228,0.105405,0.126512,0.102172,0.10732
MNAR,0.7,0.12784,0.111569,0.112435,0.102188,0.17297,0.116543,0.117035,0.111522,0.116976
MCAR,0.1,0.0662117,0.114259,0.0852719,0.0763697,0.108966,0.105667,0.117029,0.0856152,0.118379
MCAR,0.3,0.0905867,0.11627,0.0989416,0.0927033,0.112567,0.112328,0.126685,0.094115,0.116023
MCAR,0.5,0.119935,0.108295,0.104159,0.0918887,0.1119,0.106853,0.130718,0.103911,0.113592
MCAR,0.7,0.121384,0.107218,0.105203,0.0969616,0.206764,0.110332,0.133616,0.127605,0.108357
MAR,0.1,0.0754968,0.0980524,0.0846442,0.0750891,0.0929672,0.0945197,0.102033,0.0873667,0.113007
MAR,0.3,0.0558281,0.0767528,0.0616847,0.0599513,0.0687008,0.0700553,0.0718473,0.0657213,0.0796162




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0188717,0.0545958,0.0292893,0.019123,0.0309516,0.0120571,0.0308701,0.0367572,0.0181888
MNAR,0.3,0.0442481,0.180024,0.0744314,0.0418344,0.109501,0.0419972,0.0734058,0.108091,0.11734
MNAR,0.5,0.0907428,0.301556,0.120827,0.114291,0.381402,0.0922548,0.13498,0.25984,0.125623
MNAR,0.7,0.0811423,0.434336,0.299281,0.265154,0.660146,0.189492,0.241604,0.434225,0.311001
MCAR,0.1,0.0134242,0.0665873,0.0310488,0.0226686,0.042289,0.013105,0.033141,0.0390611,0.0202623
MCAR,0.3,0.0641369,0.206779,0.0873659,0.064261,0.130557,0.0465159,0.0895061,0.122988,0.0777901
MCAR,0.5,0.110639,0.307588,0.121911,0.120058,0.20649,0.0825141,0.132143,0.270707,0.205826
MCAR,0.7,0.071873,0.419067,0.180399,0.252974,0.674087,0.141922,0.235398,0.490449,0.196873
MAR,0.1,0.00878121,0.0293911,0.0154653,0.00878162,0.021026,0.00627876,0.0143403,0.0190575,0.0127796
MAR,0.3,0.0162941,0.0713557,0.0315902,0.0244034,0.0419581,0.0164696,0.0348708,0.0422619,0.0487888




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0,-0.00244898,-0.00163265,-0.00653061,-0.0179592,-0.0171429,-0.0146939,0.00326531,0.0179592
MNAR,0.3,0.00734694,-0.0342857,-0.0171429,-0.00653061,-0.00244898,-0.0195918,0.00244898,-0.0269388,-0.0138776
MNAR,0.5,0.0220408,-0.00163265,0.0302041,0.0302041,0.0391837,0.0391837,0.0261224,-0.0106122,0.00734694
MNAR,0.7,0.0310204,0.0310204,0.00571429,0.0277551,0.0138776,0.00816327,0.0408163,0.00571429,0.0318367
MCAR,0.1,0.00571429,-0.00571429,0.00653061,-0.00653061,-0.00244898,-0.0130612,0.0106122,-0.00979592,0.0204082
MCAR,0.3,-0.00163265,0.00489796,-0.0106122,0.00816327,0.000816327,0.0122449,0.0155102,-0.000816327,0.0179592
MCAR,0.5,0.0155102,-0.00408163,0.00163265,0.0244898,-0.00408163,0.00816327,0.0138776,0.0277551,0.0130612
MCAR,0.7,0.0195918,0.0138776,0.0155102,0.0204082,0.00653061,0.00571429,0.0253061,0.0163265,0.0620408
MAR,0.1,-0.00326531,-0.00653061,-0.0122449,-0.0106122,0.0114286,-0.00816327,0.00408163,-0.00326531,0.00571429
MAR,0.3,-0.0138776,-0.0212245,0.0228571,0.00163265,0.00979592,0.0220408,0.0155102,-0.00571429,0.0114286


## Yeast Data Set


In [6]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data",sep="\s+", header = None)

df = df.drop(columns = [0])

for col in [9]:
    df[col] = LabelEncoder().fit_transform(df[col])

df

,1,2,3,4,5,6,7,8,9
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,6
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,6
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,6
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,7
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,6
...,...,...,...,...,...,...,...,...,...
1479,0.81,0.62,0.43,0.17,0.5,0.0,0.53,0.22,4
1480,0.47,0.43,0.61,0.40,0.5,0.0,0.48,0.47,7
1481,0.67,0.57,0.36,0.19,0.5,0.0,0.56,0.22,4
1482,0.43,0.40,0.60,0.16,0.5,0.0,0.53,0.39,7


In [7]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("yeast", X_raw, y)

benchmark took  1753.116238117218
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.125945,0.138525,0.124456,0.124563,0.130307,0.157895,0.140038,0.125815,0.14437
MNAR,0.3,0.122669,0.120924,0.138543,0.474408,0.143974,0.146723,0.14805,0.116743,0.211633
MNAR,0.5,0.143019,0.117326,0.13094,0.390336,0.161258,0.142737,0.199797,0.121093,0.170488
MNAR,0.7,0.131071,0.116965,0.189294,0.129047,0.270225,0.143452,0.243708,0.13265,0.189738
MCAR,0.1,0.102408,0.110117,0.105621,0.102596,0.112878,0.134744,0.117223,0.103615,0.180807
MCAR,0.3,0.12223,0.11964,0.117849,0.114406,0.12425,0.141791,0.14305,0.116015,0.149102
MCAR,0.5,0.227462,0.11546,0.132303,0.15258,0.148788,0.145145,0.166237,0.116336,0.206251
MCAR,0.7,0.123386,0.118576,0.1414,0.121886,0.194473,0.149475,0.227563,0.226918,0.125716
MAR,0.1,0.128327,0.139384,0.130077,0.135631,0.168567,0.152255,0.276507,0.130812,0.241397
MAR,0.3,0.1234,0.132449,0.130485,0.122758,0.135329,0.152172,0.139881,0.126263,0.200659




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0435414,0.0672621,0.0437175,0.0421781,0.0452205,0.0243323,0.0377831,0.0509687,0.0464627
MNAR,0.3,0.0734489,0.17509,0.114883,0.167717,0.176139,0.0706597,0.113106,0.140562,0.198684
MNAR,0.5,0.097977,0.294353,0.139545,0.251447,0.346287,0.126023,0.266176,0.285289,0.187895
MNAR,0.7,0.218443,0.40494,0.310635,0.298272,1.10663,0.175972,0.630418,0.50876,0.248995
MCAR,0.1,0.0376662,0.0590753,0.0411014,0.0379584,0.0438002,0.0204809,0.0355375,0.0471644,0.034616
MCAR,0.3,0.0693771,0.175408,0.0990736,0.103725,0.149387,0.0636714,0.10563,0.148063,0.10094
MCAR,0.5,0.207005,0.289849,0.143499,0.207843,0.329727,0.114058,0.209622,0.269919,0.338447
MCAR,0.7,0.242141,0.404758,0.230241,0.279569,0.596333,0.176029,0.588903,0.768945,0.263097
MAR,0.1,0.0258716,0.0386576,0.0264184,0.0226136,0.0294993,0.0158894,0.037025,0.0286939,0.0193067
MAR,0.3,0.058119,0.0960556,0.0533997,0.056955,0.0821483,0.0385997,0.0484831,0.0703301,0.0552522




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.333842,0.538279,-0.408829,-0.163807,-0.0273539,-0.270486,-0.220029,-0.401102,-0.105056
MNAR,0.3,-0.575868,1.09426,0.884959,0.832516,0.136983,0.834459,1.21054,0.529605,-0.644091
MNAR,0.5,1.09163,1.4047,0.546397,0.0979955,0.486862,0.127368,1.48724,1.0671,0.831851
MNAR,0.7,3.75583,3.79441,2.2919,2.81196,1.00333,-0.0093457,-0.304572,2.04517,-0.137345
MCAR,0.1,0.038646,0.345303,0.305334,0.223507,0.272635,-0.300882,0.582269,0.149741,-0.925991
MCAR,0.3,0.086084,1.40627,-0.185991,-0.845075,0.585701,-0.100261,-0.058128,-0.57747,-1.38511
MCAR,0.5,1.22857,3.90463,1.79153,-0.288388,-0.336008,0.210883,2.17748,1.62608,1.43795
MCAR,0.7,4.63469,6.47155,5.22561,1.36868,0.49681,1.15989,1.67849,2.66846,0.505834
MAR,0.1,0.153869,0.819711,0.579431,0.215426,0.249278,0.0124508,0.163121,0.438526,0.115126
MAR,0.3,-0.822019,0.514985,-0.143287,0.205721,-0.0106678,0.438263,0.409718,0.833489,0.01622


## Diabetes


In [4]:
from sklearn.datasets import load_diabetes

X, y = load_diabetes(as_frame = True, return_X_y = True)

X

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018118,0.044485
439,0.041708,0.050680,-0.015906,0.017282,-0.037344,-0.013840,-0.024993,-0.011080,-0.046879,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044528,-0.025930


In [5]:
evaluate_dataset_repeated("diabetes", X, y)

benchmark took  1108.6461713314056
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.222286,0.258611,0.207309,0.192466,0.222272,0.317546,0.245276,0.194138,0.350423
MNAR,0.3,0.238452,0.253043,0.215814,0.203783,0.222732,0.289086,0.243985,0.20139,0.273777
MNAR,0.5,0.303333,0.250482,0.240296,0.243825,0.24251,0.267965,0.262982,0.273621,0.267322
MNAR,0.7,0.344291,0.253511,0.276074,0.249656,0.284133,0.249511,0.282974,0.391341,0.277253
MCAR,0.1,0.222616,0.229916,0.205023,0.189248,0.225382,0.268853,0.221326,0.191103,0.247555
MCAR,0.3,0.151096,0.173332,0.142765,0.149044,0.150582,0.148372,0.166595,0.133709,0.207213
MCAR,0.5,0.195569,0.187574,0.173077,0.166205,0.172847,0.166627,0.179487,0.176458,0.286744
MCAR,0.7,0.323852,0.244087,0.261368,0.222087,0.30203,0.286302,0.265912,0.503505,0.263141
MAR,0.1,0.330602,0.316601,0.302709,0.275914,0.295659,0.385305,0.319713,0.276921,0.333016
MAR,0.3,0.126193,0.182665,0.132853,0.125831,0.157598,0.150815,0.163263,0.128699,0.196782




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0469299,0.153246,0.0729738,0.0637556,0.0741792,0.102312,0.0712301,0.0657199,0.136464
MNAR,0.3,0.109953,0.481376,0.237646,0.176171,0.256243,0.263264,0.230681,0.218274,0.254482
MNAR,0.5,0.338342,0.790299,0.272211,0.348411,0.553691,0.399136,0.34895,0.545561,0.544419
MNAR,0.7,0.559856,1.101,0.488051,0.531964,0.776831,0.519599,0.547205,0.927716,0.724568
MCAR,0.1,0.0384297,0.142666,0.0691897,0.0604959,0.0827683,0.0594415,0.0698783,0.0662842,0.0974609
MCAR,0.3,0.0818189,0.334039,0.117347,0.0875486,0.149664,0.102048,0.124006,0.124035,0.124664
MCAR,0.5,0.177118,0.610403,0.200746,0.2027,0.365513,0.223856,0.250454,0.341473,0.230906
MCAR,0.7,0.309347,1.03505,0.354639,0.540964,0.98299,0.566729,0.476189,1.87154,0.567891
MAR,0.1,0.0466857,0.102145,0.0592968,0.0537279,0.0597768,0.0456732,0.0629418,0.0543772,0.0756726
MAR,0.3,0.0576067,0.166087,0.0629178,0.0608146,0.0994516,0.0609499,0.0976604,0.0726027,0.0575837




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,-325.036,-864.114,-339.023,95.8308,-420.316,149.551,200.118,170.937,-256.16
MNAR,0.3,-146.815,119.157,-114.473,171.962,1004.1,-355.237,-276.54,344.523,394.528
MNAR,0.5,1051.91,1059.82,587.53,-35.1043,91.2758,-467.944,-236.901,-735.295,-444.136
MNAR,0.7,639.678,1138.42,811.315,-687.703,737.055,364.014,586.555,776.966,356.153
MCAR,0.1,-121.636,-379.81,-295.321,-599.168,-61.5442,-174.726,301.752,174.434,140.785
MCAR,0.3,-376.672,104.626,-339.398,-540.824,334.725,-135.747,339.069,468.572,34.6182
MCAR,0.5,-132.582,1376.02,373.641,808.288,239.636,388.16,1880.01,503.429,215.095
MCAR,0.7,981.332,4824,2133.63,2609.51,1066.92,1663.97,1607.16,2318.03,360.839
MAR,0.1,304.113,-366.073,-299.474,-566.946,9.13203,-195.649,-255.315,110.042,98.1373
MAR,0.3,402.979,276.052,-362.258,791.25,83.7996,284.527,115.899,-869.057,707.78


## Iris


In [30]:
from sklearn.datasets import load_iris

X, y = load_iris(as_frame = True, return_X_y = True)

X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [31]:
evaluate_dataset_repeated("iris", X, y)

benchmark took  624.8973932266235
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0919293,0.282121,0.0946282,0.099976,0.146684,0.132792,0.260574,0.104095,0.512621
MNAR,0.3,0.131491,0.303971,0.123758,0.157142,0.201606,0.20669,0.293128,0.21142,0.315086
MNAR,0.5,0.158088,0.271179,0.201482,0.199724,0.170404,0.188705,0.294177,1.64373,0.253194
MNAR,0.7,0.280228,0.293407,0.350004,0.26326,0.369343,0.268158,0.4408,0.391555,0.259246
MCAR,0.1,0.0932662,0.278229,0.0933938,0.130936,0.167593,0.144868,0.196726,0.126437,0.300753
MCAR,0.3,0.152188,0.259884,0.135367,0.15188,0.165065,0.169694,0.24125,0.706153,0.324414
MCAR,0.5,0.206462,0.261032,0.1961,0.172343,0.18018,0.180866,0.311209,0.246272,0.415818
MCAR,0.7,0.297928,0.265525,0.295499,0.264472,0.354899,0.242114,0.387087,1.06572,0.299356
MAR,0.1,0.0917499,0.317273,0.100599,0.0913414,0.172075,0.112927,0.11803,0.0920906,0.250999
MAR,0.3,0.0848179,0.29628,0.0949496,0.084586,0.122446,0.102692,0.104242,0.0873698,0.296225




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0210717,0.0899623,0.0185858,0.0243216,0.039061,0.0241029,0.0400233,0.0238921,0.113547
MNAR,0.3,0.060853,0.283129,0.0578495,0.0663839,0.149256,0.100815,0.170606,0.102497,0.211133
MNAR,0.5,0.129296,0.468269,0.152667,0.187818,0.187359,0.18846,0.313454,1.13231,0.273669
MNAR,0.7,0.377128,0.719673,0.55005,0.434189,0.758675,0.451447,0.910111,0.846361,0.423145
MCAR,0.1,0.0267747,0.102801,0.0247908,0.0177405,0.0469313,0.0186538,0.0408821,0.018539,0.0759934
MCAR,0.3,0.0547831,0.244913,0.0556991,0.0561104,0.0905689,0.0624184,0.113931,0.16632,0.129447
MCAR,0.5,0.1465,0.435214,0.119289,0.142189,0.205194,0.148238,0.345153,0.240253,0.273715
MCAR,0.7,0.376703,0.634941,0.376121,0.472151,0.720545,0.264104,0.699813,1.8233,0.410056
MAR,0.1,0.00865196,0.0563895,0.0121622,0.00855977,0.0268225,0.0121305,0.0152241,0.00902464,0.0218828
MAR,0.3,0.0202249,0.1654,0.0306559,0.0202898,0.0522316,0.025775,0.0285332,0.0213648,0.0756242




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0,0,0,0,-0.0263158,0.0526316,0,0,0.0263158
MNAR,0.3,-0.0263158,0,0.0526316,0,0.0263158,0.0263158,0,0,0
MNAR,0.5,0.131579,0.0263158,0.0263158,-0.0263158,0,0.0263158,0,0.105263,0.131579
MNAR,0.7,0.0263158,0.0263158,0.0526316,0.0526316,0.0263158,0.0526316,0.131579,0.0263158,0.131579
MCAR,0.1,0,0,0,0.0263158,0,0,0.0526316,0,0
MCAR,0.3,0,0,0.0526316,0,0,0,0,0,0
MCAR,0.5,0.0263158,0,0,-0.0263158,0,0,0.0526316,0.0263158,0
MCAR,0.7,0,-0.0263158,-0.0263158,0.0526316,0.105263,0.0263158,0.157895,0.0526316,0.263158
MAR,0.1,0,0,0,0.0263158,0.0263158,0,0,0,0
MAR,0.3,0,0,0,0,0,0.0263158,0,0.0263158,-0.0263158


## Ionosphere

In [37]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data",sep=",", header = None)

for col in df.columns:
    if df[col].dtype == "object":
        df[col] = LabelEncoder().fit_transform(df[col])

last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

X_raw

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,...,0.95378,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,...,0.94520,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,...,0.93988,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,...,0.91050,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243


In [38]:
evaluate_dataset_repeated("ionosphere", X_raw, y)

benchmark took  1917.9324901103973
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.209204,0.291377,0.199728,0.207001,0.185171,0.194078,0.20799,0.197459,0.311427
MNAR,0.3,0.189525,0.274545,0.190141,0.198944,0.191496,0.185572,0.213256,0.192449,0.395447
MNAR,0.5,0.212316,0.279566,0.205023,0.271025,0.249525,0.1955,0.233657,0.322517,0.374701
MNAR,0.7,0.217368,0.27384,0.222846,0.25545,0.245441,0.206327,0.229198,0.302272,0.316817
MCAR,0.1,0.180681,0.2452,0.177016,0.187026,0.189077,0.181759,0.208818,0.179541,0.333954
MCAR,0.3,0.187909,0.25963,0.183174,0.317043,0.183456,0.181684,0.209281,0.194794,0.349804
MCAR,0.5,0.237678,0.278836,0.2244,0.269529,0.237988,0.222129,0.250087,1.11446,0.319905
MCAR,0.7,0.221159,0.263857,0.22626,0.244391,0.272551,0.204927,0.227947,0.527284,0.299344
MAR,0.1,0.169897,0.279668,0.162759,0.170523,0.161948,0.154386,0.218448,0.216618,0.419458
MAR,0.3,0.133719,0.254525,0.13024,0.14857,0.149864,0.139004,0.162755,0.181993,0.26815




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0730447,0.20433,0.0698394,0.0558253,0.0647453,0.0595032,0.0785876,0.0566561,0.113844
MNAR,0.3,0.163902,0.514625,0.169704,0.125487,0.178542,0.137109,0.21755,0.148933,0.435248
MNAR,0.5,0.323854,0.889931,0.329741,0.20995,0.463328,0.304772,0.414043,0.469422,0.428889
MNAR,0.7,0.411562,1.18799,0.400305,0.339248,0.455013,0.525899,0.576447,1.03787,0.389213
MCAR,0.1,0.0551904,0.135309,0.0533268,0.0419861,0.0506977,0.0437323,0.0701101,0.0455715,0.0636561
MCAR,0.3,0.152506,0.461943,0.160711,0.155368,0.146267,0.128126,0.184971,0.143856,0.181839
MCAR,0.5,0.31332,0.800732,0.315015,0.265836,0.307956,0.309417,0.431437,1.89576,0.348744
MCAR,0.7,0.385213,1.14998,0.392754,0.401912,0.79976,0.49414,0.474882,1.93578,0.599372
MAR,0.1,0.0375402,0.101325,0.0316065,0.030812,0.0346143,0.0250714,0.0439528,0.0302075,0.063309
MAR,0.3,0.058258,0.25081,0.0605479,0.0560337,0.0792817,0.0511913,0.0655606,0.0806191,0.0863536




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0,0,0,0.0113636,0,0,0,0,-0.0113636
MNAR,0.3,-0.0113636,-0.0113636,-0.0113636,-0.0113636,0,0.0113636,0,0,0
MNAR,0.5,-0.0113636,0,-0.0227273,-0.0113636,-0.0113636,-0.0113636,0.0227273,0,-0.0113636
MNAR,0.7,0.0113636,-0.0113636,-0.0113636,-0.0113636,0.0113636,0,0,-0.0227273,0.0113636
MCAR,0.1,0,0.0113636,0,-0.0113636,0,-0.0113636,0.0113636,-0.0113636,-0.0113636
MCAR,0.3,-0.0227273,0,0.0227273,0,0.0113636,0,0.0113636,0,-0.0227273
MCAR,0.5,0.0113636,-0.0113636,0,0,0.0227273,0,-0.0113636,-0.0113636,0.0113636
MCAR,0.7,-0.0113636,0,-0.0340909,0.0113636,-0.0113636,0,-0.0227273,0.0227273,0.0113636
MAR,0.1,-0.0113636,0.0113636,0,0,0,0,0.0113636,0.0227273,0
MAR,0.3,0,0.0113636,0,-0.0113636,0,-0.0113636,-0.0113636,0,0


## Libras

In [39]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/libras/movement_libras.data",sep=",", header = None)

last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

X_raw

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,0.79691,0.38194,0.79691,0.37731,0.79884,0.37731,0.79497,0.37731,0.77563,0.35417,...,0.34043,0.51389,0.39845,0.42593,0.47389,0.36111,0.55899,0.31250,0.63830,0.29398
1,0.67892,0.27315,0.68085,0.27315,0.68085,0.27315,0.68085,0.27315,0.67892,0.26852,...,0.19536,0.57407,0.17795,0.63657,0.17215,0.67361,0.17021,0.69213,0.17215,0.69213
2,0.72147,0.23611,0.72340,0.23611,0.72340,0.23611,0.72340,0.23611,0.72340,0.23611,...,0.52031,0.30556,0.59768,0.25926,0.67118,0.25231,0.73501,0.26620,0.78143,0.27778
3,0.56480,0.32407,0.56286,0.32407,0.56093,0.32407,0.55899,0.32407,0.55899,0.32407,...,0.20503,0.49074,0.26306,0.42361,0.33269,0.34722,0.41006,0.28009,0.49130,0.24306
4,0.67118,0.38426,0.67118,0.38657,0.67311,0.38657,0.67311,0.38426,0.67311,0.37963,...,0.46422,0.76389,0.44101,0.64120,0.45068,0.54167,0.47776,0.44213,0.53191,0.34259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0.65957,0.79167,0.65764,0.78704,0.65957,0.78935,0.65957,0.78704,0.65764,0.78241,...,0.58221,0.55093,0.57253,0.53935,0.56286,0.53241,0.55126,0.52546,0.54159,0.52083
356,0.64023,0.71991,0.64217,0.71759,0.64217,0.71759,0.64217,0.71759,0.64217,0.71991,...,0.56480,0.50463,0.55513,0.50231,0.54352,0.49769,0.53191,0.49537,0.52031,0.49306
357,0.61122,0.75926,0.61122,0.75694,0.61315,0.75694,0.61122,0.75694,0.61122,0.75926,...,0.49710,0.54167,0.48549,0.53472,0.47195,0.52546,0.45841,0.52083,0.44487,0.51620
358,0.65957,0.79167,0.65764,0.78704,0.65957,0.78935,0.65957,0.78704,0.65764,0.78241,...,0.58221,0.55093,0.57253,0.53935,0.56286,0.53241,0.55126,0.52546,0.54159,0.52083


In [40]:
evaluate_dataset_repeated("libras", X_raw, y)

benchmark took  4423.736758232117
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.00597045,0.227603,0.0268882,0.00532613,0.043374,0.0670944,0.169932,0.0302685,0.253104
MNAR,0.3,0.0128165,0.205886,0.0327947,0.00686471,0.0395187,0.0748147,0.168356,0.0780116,0.227688
MNAR,0.5,0.0160053,0.220466,0.0269913,0.0106884,0.0511445,0.0851699,0.169098,0.247412,0.250603
MNAR,0.7,0.0384381,0.238201,0.0404254,0.0124571,0.090525,0.110411,0.185635,0.67324,0.456889
MCAR,0.1,0.00930634,0.21388,0.0280739,0.00822892,0.0403061,0.0676231,0.168542,0.0474509,0.26247
MCAR,0.3,0.0182442,0.213301,0.0337434,0.0111219,0.0447292,0.0828341,0.187481,0.0678988,0.315575
MCAR,0.5,0.0372408,0.214652,0.0323526,0.0142206,0.0520377,0.0919526,0.205662,0.0671369,0.260965
MCAR,0.7,0.0361814,0.221784,0.0338861,0.0114519,0.0436829,0.100265,0.169362,0.196355,0.303032
MAR,0.1,0.00976967,0.241428,0.0292047,0.0075508,0.0474716,0.0712557,0.18526,0.062332,0.23722
MAR,0.3,0.00865522,0.223621,0.0290927,0.00704254,0.0350005,0.0757634,0.184116,0.084191,0.247742




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.00302384,0.164587,0.0159478,0.00263313,0.0188078,0.0277497,0.0832056,0.0164415,0.0719869
MNAR,0.3,0.0128565,0.409244,0.0417008,0.00819682,0.0437936,0.0717356,0.212503,0.142715,0.246437
MNAR,0.5,0.0227998,0.754192,0.0592139,0.0163607,0.102271,0.176206,0.274098,0.744998,0.436739
MNAR,0.7,0.0463762,1.10614,0.108981,0.0230128,0.301284,0.327601,0.451259,2.78907,1.17691
MCAR,0.1,0.0039217,0.153767,0.0153798,0.00355836,0.0180954,0.0265448,0.0549545,0.0339994,0.0637718
MCAR,0.3,0.0154556,0.410954,0.0400161,0.00935382,0.0483854,0.0865823,0.183654,0.125181,0.147873
MCAR,0.5,0.0370348,0.719801,0.0642709,0.0189107,0.077672,0.169062,0.377799,0.229176,0.383312
MCAR,0.7,0.0533033,1.04135,0.0915407,0.0199771,0.120274,0.296977,0.43433,0.832925,0.341656
MAR,0.1,0.00183628,0.0771448,0.00708963,0.0014221,0.00988785,0.0139486,0.0395718,0.0189542,0.0309466
MAR,0.3,0.00438009,0.227306,0.0225898,0.00363279,0.0204274,0.0466378,0.128594,0.0779752,0.126613




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.16743,0.246539,0.126046,-0.471958,-0.330055,0.0097752,-0.52716,-0.215851,-0.0871481
MNAR,0.3,-0.0974554,-0.958906,0.9221,0.180154,0.725792,-0.273634,-0.707905,1.85756,-0.453698
MNAR,0.5,-0.814633,-0.073992,0.149998,-0.330505,-0.727585,0.0400245,-0.499224,0.240871,0.505351
MNAR,0.7,1.30015,0.583805,-0.167947,1.52382,1.39823,0.920522,1.78556,1.04382,1.8355
MCAR,0.1,-0.0456665,-0.161116,0.043144,-0.0954853,-0.399378,-0.586012,0.320111,0.598078,-0.521285
MCAR,0.3,-0.191272,-0.153824,0.622651,0.130794,0.146245,-0.032758,-1.29053,0.228712,0.822322
MCAR,0.5,-0.570244,-0.197529,-0.0896086,0.364387,-0.00732754,0.548637,-0.478599,-1.89444,0.627733
MCAR,0.7,-1.17696,0.461201,-0.0413185,1.23052,-0.0766665,1.4917,0.29347,0.141694,0.465727
MAR,0.1,0.0742414,0.154804,0.14206,-0.127304,-0.695917,-0.361834,-0.609476,1.83469,0.512793
MAR,0.3,-0.0834833,0.61689,0.273414,-0.151967,0.811037,0.328283,0.471039,-0.304372,-0.321424


## Parkinsons

In [41]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data",sep=",")

for col in df.columns:
    if df[col].dtype == "object":
        df[col] = LabelEncoder().fit_transform(df[col])
  
df = df.drop(columns = ["name"])

last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

X_raw

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,MDVP:APQ,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2
0,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,...,0.02971,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442
1,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,...,0.04368,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855
2,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,...,0.03590,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259
3,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,...,0.03772,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554
4,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,...,0.04465,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,174.188,230.978,94.261,0.00459,0.00003,0.00263,0.00259,0.00790,0.04087,0.405,...,0.02745,0.07008,0.02764,19.517,0,0.448439,0.657899,-6.538586,0.121952,2.657476
191,209.516,253.017,89.488,0.00564,0.00003,0.00331,0.00292,0.00994,0.02751,0.263,...,0.01879,0.04812,0.01810,19.147,0,0.431674,0.683244,-6.195325,0.129303,2.784312
192,174.688,240.005,74.287,0.01360,0.00008,0.00624,0.00564,0.01873,0.02308,0.256,...,0.01667,0.03804,0.10715,17.883,0,0.407567,0.655683,-6.787197,0.158453,2.679772
193,198.764,396.961,74.904,0.00740,0.00004,0.00370,0.00390,0.01109,0.02296,0.241,...,0.01588,0.03794,0.07223,19.020,0,0.451221,0.643956,-6.744577,0.207454,2.138608


In [42]:
evaluate_dataset_repeated("parkinsons", X_raw, y)

benchmark took  1211.97238779068
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0890606,0.2304,0.111856,0.083093,0.117586,0.113535,0.181958,0.57725,0.328346
MNAR,0.3,0.142736,0.198795,0.120254,0.148335,0.159156,0.122953,0.167498,2.13311,0.254418
MNAR,0.5,0.244131,0.196013,0.137012,0.115536,0.164533,0.133859,0.180476,0.149977,0.207645
MNAR,0.7,0.322112,0.228815,0.209529,0.20174,0.237647,0.189029,0.206098,0.430777,0.392682
MCAR,0.1,0.180426,0.244357,0.126251,0.13125,0.169321,0.177461,0.222921,0.130914,0.301245
MCAR,0.3,0.132216,0.197436,0.134618,0.137593,0.145493,0.126998,0.190965,0.154026,0.273961
MCAR,0.5,0.226818,0.184863,0.114635,0.113042,0.145174,0.111144,0.153206,0.121262,0.281356
MCAR,0.7,0.242256,0.186818,0.105433,0.0936472,0.125643,0.110484,0.167615,0.179759,0.267304
MAR,0.1,0.160569,0.258047,0.114557,0.0857097,0.148411,0.117354,0.142507,0.145301,0.33572
MAR,0.3,0.140794,0.271734,0.127842,0.161645,0.173787,0.168095,0.306289,0.159542,0.382653




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0284123,0.128299,0.0400002,0.0234548,0.0438444,0.0469688,0.0721909,0.0968275,0.124893
MNAR,0.3,0.0851214,0.329462,0.0959019,0.0933182,0.161648,0.104829,0.167774,0.593068,0.131799
MNAR,0.5,0.2725,0.556174,0.164333,0.117388,0.330376,0.204228,0.314869,0.264436,0.392468
MNAR,0.7,0.473295,0.90996,0.2621,0.387986,0.485932,0.346705,0.582846,1.31452,0.517177
MCAR,0.1,0.0422224,0.148358,0.0382494,0.0484986,0.0588281,0.0503467,0.0932824,0.0529584,0.0969054
MCAR,0.3,0.105589,0.352384,0.0982199,0.0660543,0.166027,0.110027,0.201611,0.180867,0.167614
MCAR,0.5,0.305249,0.551018,0.147409,0.117034,0.271983,0.165055,0.299297,0.178498,0.332611
MCAR,0.7,0.317112,0.772689,0.175802,0.129547,0.30785,0.263345,0.417958,0.607813,0.376596
MAR,0.1,0.0162984,0.0774727,0.0242211,0.0151647,0.0290345,0.0225685,0.0280444,0.0270764,0.0735967
MAR,0.3,0.0244219,0.229759,0.0562225,0.0798099,0.0709044,0.0481892,0.162216,0.0930139,0.142756




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,-4.53602e-05,-2.54928e-05,-1.36845e-05,-2.37511e-05,-6.56188e-06,-6.11687e-05,0.000284704,-0.000114325,-6.97161e-05
MNAR,0.3,6.11621e-05,-1.30374e-05,-3.56015e-05,-5.83205e-05,2.50004e-06,-4.47471e-05,7.86432e-05,-8.43046e-05,6.9031e-06
MNAR,0.5,8.61873e-05,0.000137562,-3.57259e-05,9.46489e-06,0.000170349,0.000237586,3.30113e-05,-0.000106233,-0.000115248
MNAR,0.7,5.82789e-05,-3.89713e-05,-0.00012714,4.10936e-05,-0.000128198,0.000126619,0.000171264,5.43204e-05,4.32175e-05
MCAR,0.1,9.54183e-05,0.000103998,-5.45388e-05,-3.4133e-06,9.78496e-05,5.3473e-05,0.000149153,-9.60158e-05,-1.50143e-06
MCAR,0.3,0.000118054,0.000252272,-1.28129e-05,0.000102005,1.09306e-05,6.75718e-05,3.70641e-05,-3.17848e-05,3.97513e-06
MCAR,0.5,8.61829e-05,-6.51153e-06,-0.000374354,0.000141412,0.000648861,-8.12563e-05,-3.31067e-05,5.60566e-05,1.67211e-05
MCAR,0.7,0.00127488,0.00077182,0.000631784,0.000281655,0.000624193,0.000731941,0.00171463,0.00221001,0.00159754
MAR,0.1,0.000194905,0.000771898,0.000156424,0.000170186,7.34145e-05,-3.16597e-05,0.000294294,0.000132043,0.000664299
MAR,0.3,6.13492e-05,-1.08125e-05,-3.83433e-05,0.000317991,-1.17542e-05,4.85208e-06,0.000130696,0.000134915,6.02756e-05


## Yacht

In [49]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00243/yacht_hydrodynamics.data",sep="\s+", header = None)

for col in df.columns:
    if df[col].dtype == "object":
        df[col] = LabelEncoder().fit_transform(df[col])
  
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

X_raw

,0,1,2,3,4,5
0,-2.3,0.568,4.78,3.99,3.17,0.125
1,-2.3,0.568,4.78,3.99,3.17,0.150
2,-2.3,0.568,4.78,3.99,3.17,0.175
3,-2.3,0.568,4.78,3.99,3.17,0.200
4,-2.3,0.568,4.78,3.99,3.17,0.225
...,...,...,...,...,...,...
303,-2.3,0.600,4.34,4.23,2.73,0.350
304,-2.3,0.600,4.34,4.23,2.73,0.375
305,-2.3,0.600,4.34,4.23,2.73,0.400
306,-2.3,0.600,4.34,4.23,2.73,0.425


In [50]:
evaluate_dataset_repeated("yacht", X_raw, y)

benchmark took  794.7721018791199
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.202852,0.27627,0.211878,0.202916,0.298276,0.216667,0.314239,0.354792,0.305664
MNAR,0.3,0.267999,0.290784,0.255564,0.251385,0.304269,0.288094,0.389442,0.310463,0.311271
MNAR,0.5,0.315799,0.300081,0.318439,0.300297,0.322627,0.291628,0.424111,0.52498,0.352448
MNAR,0.7,0.405747,0.30266,0.331244,0.305716,0.454943,0.324035,0.477927,1.15541,0.321916
MCAR,0.1,0.193223,0.293661,0.207275,0.186602,0.309349,0.205442,0.288105,0.203795,0.307547
MCAR,0.3,0.263472,0.29797,0.248086,0.249971,0.311162,0.283475,0.36527,0.272554,0.485081
MCAR,0.5,0.369147,0.293797,0.310994,0.289414,0.308071,0.316965,0.398916,0.518725,0.313668
MCAR,0.7,0.363321,0.293254,0.343793,0.294738,0.369478,0.323996,0.425454,0.643062,0.316043
MAR,0.1,0.162762,0.262049,0.169074,0.204907,0.255783,0.22073,0.316681,0.166298,0.335187
MAR,0.3,0.240119,0.296492,0.246466,0.240243,0.319296,0.293833,0.366733,0.339064,0.405404




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0687025,0.128026,0.0703651,0.072826,0.095353,0.0471554,0.0776621,0.091351,0.0923593
MNAR,0.3,0.194741,0.383156,0.205016,0.247753,0.273518,0.175835,0.185696,0.323591,0.25898
MNAR,0.5,0.322589,0.694451,0.372361,0.484361,0.561805,0.349049,0.498425,0.701831,0.507715
MNAR,0.7,0.68401,0.940308,0.444571,0.684531,1.25721,0.536657,0.856765,2.9184,0.660506
MCAR,0.1,0.0659395,0.139183,0.0627872,0.0679538,0.125564,0.0487358,0.0784655,0.0754322,0.0975791
MCAR,0.3,0.201204,0.3925,0.201993,0.23554,0.311673,0.182418,0.174466,0.26624,0.468162
MCAR,0.5,0.349453,0.63746,0.437728,0.439516,0.502691,0.32488,0.398505,0.84877,0.452103
MCAR,0.7,0.488312,0.868417,0.462052,0.655995,0.866057,0.42534,0.636018,1.61036,0.586264
MAR,0.1,0.0241883,0.0514708,0.0264256,0.0283568,0.0284972,0.0221771,0.0333215,0.0263599,0.0296883
MAR,0.3,0.109989,0.182162,0.129693,0.11462,0.134917,0.079134,0.0982042,0.141455,0.102126




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.560147,10.2819,7.28674,0.320554,0.380914,5.32011,0.430372,0.470228,1.32645
MNAR,0.3,9.19342,77.972,4.28152,4.75526,8.84337,13.9877,21.6455,5.26481,3.43273
MNAR,0.5,92.3855,50.415,8.62762,41.6052,14.4914,24.608,25.8022,14.9294,11.3086
MNAR,0.7,173.041,334.608,90.7051,244.348,34.1713,244.674,62.2338,44.496,133.985
MCAR,0.1,0.578213,5.34443,-0.530345,0.41303,0.291193,5.14945,0.683231,0.212448,0.542624
MCAR,0.3,12.2877,15.6947,10.0902,9.14795,3.86137,73.3306,46.687,3.44542,2.8005
MCAR,0.5,170.741,94.1908,98.5973,26.2043,108.657,17.0423,57.3195,12.7561,25.4555
MCAR,0.7,141.778,58.4954,104.92,52.3013,94.6341,51.996,65.008,188.094,39.0122
MAR,0.1,0.28446,0.699732,0.0588037,-0.207228,0.143641,7.73835,0.404034,0.229294,0.383534
MAR,0.3,3.75348,8.46226,1.71259,0.556368,2.94745,5.4365,13.3007,3.30434,6.90174


## Spam

In [16]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data")

for col in df.columns:
    if df[col].dtype == "object":
        df[col] = LabelEncoder().fit_transform(df[col])
  
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

X_raw

,0,0.64,0.64.1,0.1,0.32,0.2,0.3,0.4,0.5,0.6,...,0.39,0.40,0.41,0.42,0.778,0.43,0.44,3.756,61,278
0,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.0,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028
1,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.0,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259
2,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191
4,0.00,0.00,0.00,0.0,1.85,0.00,0.00,1.85,0.00,0.00,...,0.0,0.000,0.223,0.0,0.000,0.000,0.000,3.000,15,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.0,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88
4596,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14
4597,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118
4598,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.0,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78


In [17]:
evaluate_dataset_repeated("spam", X_raw, y)

benchmark took  13310.196274280548
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0409359,0.067968,0.0465604,0.0645353,0.0608444,0.0645095,0.0607957,0.0642364,0.0708463
MNAR,0.3,0.0557717,0.0649066,0.0592677,0.0773353,0.0630593,0.0673035,0.060696,0.0594236,0.095136
MNAR,0.5,0.0561307,0.0572109,0.0620238,0.0619511,0.0572722,0.0603496,0.0494969,0.0572,0.0590884
MNAR,0.7,0.0706562,0.0588481,0.0874864,0.0594881,0.057836,0.0667081,0.054993,0.0588241,0.0617636
MCAR,0.1,0.0418241,0.052967,0.0455159,0.0471419,0.0528494,0.0639702,0.0497993,0.0492838,0.0562084
MCAR,0.3,0.0543759,0.0714276,0.0615711,0.0660904,0.0713415,0.0775451,0.0702769,0.0697392,0.082143
MCAR,0.5,0.0442661,0.051523,0.0447208,0.0602237,0.0505809,0.0619872,0.0512665,0.0515942,0.0532618
MCAR,0.7,0.0643827,0.0653644,0.0663363,0.0629029,0.0654471,0.0715416,0.0624065,0.0652291,0.0675498
MAR,0.1,0.0723504,0.139664,0.0836756,0.0768132,0.13874,0.119076,0.116872,0.137757,0.149876
MAR,0.3,0.0450935,0.0753061,0.0512786,0.0520956,0.0585283,0.0697731,0.0502938,0.075187,0.0781063




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.00663994,0.0228213,0.0117784,0.0151169,0.0187658,0.00894594,0.0142909,0.0211019,0.01686
MNAR,0.3,0.0251606,0.0760671,0.037093,0.0538855,0.0640347,0.0327558,0.0470536,0.0693668,0.0468508
MNAR,0.5,0.0328121,0.0850253,0.0585448,0.0721689,0.0668953,0.0572724,0.0568174,0.0832573,0.0543868
MNAR,0.7,0.0549566,0.136264,0.103586,0.100585,0.110192,0.091439,0.0853291,0.125354,0.0874756
MCAR,0.1,0.00581494,0.0177025,0.00967346,0.0106325,0.0180806,0.00989871,0.0104062,0.0148293,0.0108021
MCAR,0.3,0.0293052,0.0780937,0.0396698,0.053371,0.0775743,0.0337893,0.0474762,0.073699,0.0465313
MCAR,0.5,0.0386724,0.0820867,0.0411932,0.0680482,0.0715751,0.0575368,0.036899,0.0704345,0.0514712
MCAR,0.7,0.0538512,0.146364,0.0857625,0.113933,0.127943,0.0826965,0.0773187,0.145522,0.0908364
MAR,0.1,0.0083444,0.0257041,0.0143556,0.0123155,0.025251,0.0126533,0.0145617,0.0256299,0.0199073
MAR,0.3,0.0138164,0.0336747,0.0182209,0.0182868,0.0259559,0.0182389,0.0157526,0.0330991,0.0244547




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.00434783,-0.000869565,-0.000869565,-0.000869565,-0.00434783,0,0,-0.00347826,0.000869565
MNAR,0.3,0.00347826,-0.00695652,0.00173913,-0.00173913,0.00608696,0.00434783,0.0104348,-0.0026087,0.00695652
MNAR,0.5,0.0104348,-0.00782609,0.00173913,0.013913,0.0130435,0.00608696,0.00608696,0.00173913,0.000869565
MNAR,0.7,0.0191304,0.0026087,0.0165217,0.00695652,0.0234783,0.00782609,0.0191304,0.0182609,-0.00347826
MCAR,0.1,-0.00695652,0,0,-0.00347826,0.00173913,0.00695652,0.000869565,0.00434783,-0.000869565
MCAR,0.3,-0.00434783,-0.0026087,0.0026087,-0.00347826,0.00782609,0.000869565,0.00521739,-0.00173913,0.00608696
MCAR,0.5,-0.00521739,0,-0.00347826,0.00434783,0.00869565,-0.00608696,0.0104348,0.00173913,-0.00521739
MCAR,0.7,0.00608696,0.00695652,0.0147826,0.00521739,0,0.00521739,0.0104348,0.000869565,0.00869565
MAR,0.1,0,-0.00173913,0.000869565,-0.0026087,-0.000869565,-0.00173913,-0.00347826,-0.00695652,-0.00173913
MAR,0.3,-0.00347826,0.00956522,0.0026087,0.00347826,-0.000869565,0.00521739,0.00347826,-0.000869565,0.00173913


## Letter dataset

In [ ]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data", header = None)

for col in df.columns:
    if df[col].dtype == "object":
        df[col] = LabelEncoder().fit_transform(df[col])
  
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

X_raw

In [ ]:
evaluate_dataset_repeated("letter", X_raw, y)

## Credit dataset

In [14]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data", header = None)

for col in df.columns:
    if df[col].dtype == "object":
        df[col] = LabelEncoder().fit_transform(df[col])
  
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

X_raw

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2,156,0.000,2,1,13,8,1.25,1,1,1,0,0,68,0
1,1,328,4.460,2,1,11,4,3.04,1,1,6,0,0,11,560
2,1,89,0.500,2,1,11,4,1.50,1,0,0,0,0,96,824
3,2,125,1.540,2,1,13,8,3.75,1,1,5,1,0,31,3
4,2,43,5.625,2,1,13,8,1.71,1,0,0,0,2,37,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,2,52,10.085,3,3,5,4,1.25,0,0,0,0,0,90,0
686,1,71,0.750,2,1,2,8,2.00,0,1,2,1,0,67,394
687,1,97,13.500,3,3,6,3,2.00,0,1,1,1,0,67,1
688,2,20,0.205,2,1,1,8,0.04,0,0,0,0,0,96,750


In [15]:
evaluate_dataset_repeated("credit", X_raw, y)

benchmark took  1593.1663987636566
RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.334275,0.361517,0.272079,0.319687,0.364834,0.423446,0.362366,0.30825,0.442752
MNAR,0.3,0.233246,0.278354,0.246158,0.252189,0.28907,0.314213,0.279259,0.245302,0.304069
MNAR,0.5,0.284697,0.280566,0.263738,0.275014,0.316097,0.306677,0.309469,0.269785,0.374415
MNAR,0.7,0.369722,0.299982,0.348977,0.312977,0.363025,0.342411,0.300323,0.378627,0.422446
MCAR,0.1,0.305853,0.337019,0.275593,0.302432,0.370974,0.396323,0.337883,0.291721,0.414457
MCAR,0.3,0.297529,0.313043,0.271617,0.283549,0.318334,0.366809,0.313285,0.276279,0.377967
MCAR,0.5,0.311112,0.290988,0.302723,0.371036,0.306203,0.324324,0.290614,0.41551,0.369469
MCAR,0.7,0.39251,0.335068,0.393609,0.352901,0.397547,0.372708,0.344181,0.481903,0.558993
MAR,0.1,0.243191,0.268932,0.245217,0.252348,0.281521,0.280242,0.272643,0.255468,0.346714
MAR,0.3,0.232939,0.301341,0.264729,0.244354,0.27751,0.336455,0.295068,0.248595,0.468584




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0.0528314,0.265609,0.112445,0.170593,0.180741,0.122574,0.147407,0.165975,0.203513
MNAR,0.3,0.225151,0.51005,0.260516,0.336584,0.338794,0.25969,0.324418,0.362568,0.340177
MNAR,0.5,0.38212,0.874868,0.385518,0.624271,0.666129,0.352364,0.524771,0.717322,0.514124
MNAR,0.7,0.509726,1.23988,0.463131,0.817642,0.997192,0.608358,0.849671,0.911384,0.780025
MCAR,0.1,0.114501,0.221554,0.122537,0.156232,0.159101,0.0831301,0.122737,0.156097,0.134723
MCAR,0.3,0.156194,0.583622,0.243468,0.368967,0.323301,0.255125,0.37303,0.375882,0.326232
MCAR,0.5,0.397137,0.878931,0.424554,0.546391,0.509838,0.352706,0.540649,0.780843,0.723056
MCAR,0.7,0.552197,1.52307,0.60016,1.02178,1.17158,0.656952,1.01287,1.5412,1.77564
MAR,0.1,0.0564977,0.0852116,0.0574416,0.0633785,0.0564478,0.0381406,0.0581266,0.0659925,0.0480358
MAR,0.3,0.0860148,0.27905,0.117583,0.154856,0.174979,0.0942063,0.161787,0.156606,0.201119




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean,sklearn_missforest,sklearn_ice,gain,sinkhorn,softimpute,miracle,miwae
MNAR,0.1,0,0.00578035,-0.00578035,-0.00578035,0,0.00578035,0,-0.0289017,0.00578035
MNAR,0.3,-0.0115607,-0.0289017,0.0115607,-0.00578035,0,-0.0115607,0.017341,-0.017341,-0.00578035
MNAR,0.5,0.0578035,0.0520231,0.0520231,0.0346821,0.0404624,0.0462428,0.0578035,0.0520231,0.0635838
MNAR,0.7,0.0404624,0,0.0231214,0.0231214,-0.0231214,0.0867052,0.0346821,0.0578035,0
MCAR,0.1,0.0115607,-0.0115607,-0.0115607,-0.0115607,0,-0.0231214,-0.017341,0.0115607,-0.0289017
MCAR,0.3,-0.0231214,-0.0231214,0.0289017,0,0,-0.00578035,-0.0115607,-0.017341,0.00578035
MCAR,0.5,-0.0231214,0.0231214,-0.0115607,0.017341,0.0115607,-0.017341,0.00578035,0.0578035,0
MCAR,0.7,0.0231214,0.0115607,0,0.0289017,0.0404624,0.00578035,0.0462428,0,-0.017341
MAR,0.1,0.017341,0.0231214,-0.017341,-0.017341,0.00578035,0,0,-0.0115607,0.00578035
MAR,0.3,0.0346821,0,0.00578035,0.0231214,-0.00578035,0.00578035,0.017341,0.017341,-0.00578035


In [ ]:
## 